In [2]:
import sys
import numpy as np
import pandas as pd

import tensorflow as tf

In [3]:
print(sys.version)
print(f'tf version : {tf.__version__}')

3.7.11 (default, Jul 27 2021, 09:42:29) [MSC v.1916 64 bit (AMD64)]
tf version : 2.7.0


# tf.data.Dataset from TFRecords

references
- https://medium.com/mostly-ai/tensorflow-records-what-they-are-and-how-to-use-them-c46bc4bbb564
- 

Benefits of Binary format:
- less space on disk
- faster read and write.
- Easy integration with multiple types of datasets such as images, text, audio, etc...
- N-gram support => store sequential data.


but how to TFRecord differ from other Binary formatS?

In [8]:
movie_name = tf.train.Feature(
    bytes_list = tf.train.BytesList(value=[b'The Shawshank Redemption', b'Fight Club'])
)
movie_rating = tf.train.Feature(
    float_list = tf.train.FloatList(value=[9.0, 9.7])
)

In [12]:
print(movie_name)
print(type(movie_name))

bytes_list {
  value: "The Shawshank Redemption"
  value: "Fight Club"
}

<class 'tensorflow.core.example.feature_pb2.Feature'>


In [13]:
movie_dict = {"movie_names":movie_name, "rating":movie_rating}

In [18]:
movies = tf.train.Features(feature=movie_dict)
print(type(movies))
print(movies)

<class 'tensorflow.core.example.feature_pb2.Features'>
feature {
  key: "movie_names"
  value {
    bytes_list {
      value: "The Shawshank Redemption"
      value: "Fight Club"
    }
  }
}
feature {
  key: "rating"
  value {
    float_list {
      value: 9.0
      value: 9.699999809265137
    }
  }
}



In [19]:
example = tf.train.Example(features=tf.train.Features(feature=movie_dict))
print(type(example))

<class 'tensorflow.core.example.example_pb2.Example'>


In [17]:
example

features {
  feature {
    key: "movie_names"
    value {
      bytes_list {
        value: "The Shawshank Redemption"
        value: "Fight Club"
      }
    }
  }
  feature {
    key: "rating"
    value {
      float_list {
        value: 9.0
        value: 9.699999809265137
      }
    }
  }
}

In [20]:
write_path = 'tfrecords/movies.tfrecord'
writer = tf.io.TFRecordWriter(write_path)
writer.write(example.SerializeToString())

# We've just written one sample of data in tfrecord representing features and rating(y)

In [21]:
data = {
    'Age': 29,
    'Movie': ['The Shawshank Redemption', 'Fight Club'],
    'Movie Ratings': [9.0, 9.7],
    'Suggestion': 'Inception',
    'Suggestion Purchased': 1.0,
    'Purchase Price': 9.99
}

print(data)

{'Age': 29, 'Movie': ['The Shawshank Redemption', 'Fight Club'], 'Movie Ratings': [9.0, 9.7], 'Suggestion': 'Inception', 'Suggestion Purchased': 1.0, 'Purchase Price': 9.99}


In [31]:
age_feature = tf.train.Feature(int64_list = tf.train.Int64List(value=[data['Age']]))

#??? why two ?
movie_feature = tf.train.Feature(
    bytes_list = tf.train.BytesList(value=[m.encode('utf-8') for m in data['Movie']]))
rating_feature = tf.train.Feature(float_list=tf.train.FloatList(value=data['Movie Ratings']))
suggest_feature = tf.train.Feature(
    bytes_list=tf.train.BytesList(value=[data['Suggestion'].encode('utf-8')]))
price_feature = tf.train.Feature(float_list=tf.train.FloatList(value=[data['Purchase Price']]))

In [33]:
features_dict = {'age': age_feature,
                 'movie':movie_feature,
                 'rating':rating_feature,
                 'suggestion':suggest_feature,
                 'price':price_feature}

In [34]:
features_dict

{'age': int64_list {
   value: 29
 },
 'movie': bytes_list {
   value: "The Shawshank Redemption"
   value: "Fight Club"
 },
 'rating': float_list {
   value: 9.0
   value: 9.699999809265137
 },
 'suggestion': bytes_list {
   value: "Inception"
 },
 'price': float_list {
   value: 9.989999771118164
 }}

In [38]:
example = tf.train.Example(
    features=tf.train.Features(feature=features_dict)
)

with tf.io.TFRecordWriter('tfrecords/customer_1.tfrecord') as writer:
    writer.write(example.SerializePartialToString())

In [36]:
print(example)

features {
  feature {
    key: "age"
    value {
      int64_list {
        value: 29
      }
    }
  }
  feature {
    key: "movie"
    value {
      bytes_list {
        value: "The Shawshank Redemption"
        value: "Fight Club"
      }
    }
  }
  feature {
    key: "price"
    value {
      float_list {
        value: 9.989999771118164
      }
    }
  }
  feature {
    key: "rating"
    value {
      float_list {
        value: 9.0
        value: 9.699999809265137
      }
    }
  }
  feature {
    key: "suggestion"
    value {
      bytes_list {
        value: "Inception"
      }
    }
  }
}



In [54]:
feature_desc_dict = {
    'age': tf.io.FixedLenFeature([], dtype=tf.int64),
    'movie': tf.io.VarLenFeature(dtype=tf.string),
    'rating': tf.io.VarLenFeature(dtype=tf.float32),
    'suggestion': tf.io.FixedLenFeature([], dtype=tf.string),
    'price': tf.io.FixedLenFeature([], dtype=tf.float32)}

In [45]:
serialized_dataset = tf.data.TFRecordDataset('tfrecords/customer_1.tfrecord')

In [55]:
serialized_dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

In [56]:
serialized_dataset.element_spec

TensorSpec(shape=(), dtype=tf.string, name=None)

In [57]:
for e in serialized_dataset:
    print(e)

tf.Tensor(b'\n\x89\x01\n\x11\n\x05price\x12\x08\x12\x06\n\x04\n\xd7\x1fA\n\x16\n\x06rating\x12\x0c\x12\n\n\x08\x00\x00\x10A33\x1bA\n\x1b\n\nsuggestion\x12\r\n\x0b\n\tInception\n\x0c\n\x03age\x12\x05\x1a\x03\n\x01\x1d\n1\n\x05movie\x12(\n&\n\x18The Shawshank Redemption\n\nFight Club', shape=(), dtype=string)


In [60]:
single_data = tf.io.parse_single_example(serialized=e, features=feature_desc_dict)

In [61]:
single_data

{'movie': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x1ae69e82148>,
 'rating': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x1ae69e82208>,
 'age': <tf.Tensor: shape=(), dtype=int64, numpy=29>,
 'price': <tf.Tensor: shape=(), dtype=float32, numpy=9.99>,
 'suggestion': <tf.Tensor: shape=(), dtype=string, numpy=b'Inception'>}

In [66]:
for name, tensor in single_data.items():
    print(f'{name} : {tensor}')
    print()

movie : SparseTensor(indices=tf.Tensor(
[[0]
 [1]], shape=(2, 1), dtype=int64), values=tf.Tensor([b'The Shawshank Redemption' b'Fight Club'], shape=(2,), dtype=string), dense_shape=tf.Tensor([2], shape=(1,), dtype=int64))

rating : SparseTensor(indices=tf.Tensor(
[[0]
 [1]], shape=(2, 1), dtype=int64), values=tf.Tensor([9.  9.7], shape=(2,), dtype=float32), dense_shape=tf.Tensor([2], shape=(1,), dtype=int64))

age : 29

price : 9.989999771118164

suggestion : b'Inception'



# petastorm

In [62]:
arr = np.random.randint(10, size=10)

In [63]:
ds = tf.data.Dataset.from_tensor_slices(arr)

In [68]:
n = 0
for e in ds:
    n += e.numpy()
    print(e)
print(n)

tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
0


In [14]:
e.numpy()

1

In [26]:
type(iter(ds))

tensorflow.python.data.ops.iterator_ops.OwnedIterator

In [27]:
type(ds)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [51]:
ds2 = tf.data.Dataset.from_tensor_slices(tf.random.uniform([4,10]))

print("- - - - - - - - ds2 - - - - - - - -")
print(tf.random.uniform([4,10]))
ds3 =(tf.random.uniform([4]),
      tf.random.uniform([4, 4], maxval=100, dtype=tf.int32))

print("- - - - - - - - ds3 - - - - - - - -")
print(ds3)
ds3 = tf.data.Dataset.from_tensor_slices(ds3)

- - - - - - - - ds2 - - - - - - - -
tf.Tensor(
[[0.47324824 0.5358292  0.68424356 0.6670543  0.39903927 0.74895215
  0.02241492 0.7940999  0.96831155 0.3296466 ]
 [0.90393615 0.21639478 0.00346696 0.00973201 0.59110105 0.6534457
  0.34532034 0.44684315 0.7754146  0.75971067]
 [0.05918646 0.92768335 0.08951044 0.5959326  0.1223489  0.64387894
  0.623575   0.88988674 0.25609267 0.93290424]
 [0.13810718 0.28943217 0.4182005  0.46913886 0.93926716 0.7865521
  0.10096097 0.37386072 0.5458963  0.89973044]], shape=(4, 10), dtype=float32)
- - - - - - - - ds3 - - - - - - - -
(<tf.Tensor: shape=(4,), dtype=float32, numpy=array([0.19898903, 0.833717  , 0.17857337, 0.19229329], dtype=float32)>, <tf.Tensor: shape=(4, 4), dtype=int32, numpy=
array([[72, 98, 32, 86],
       [80,  9, 44,  6],
       [70, 44, 42, 35],
       [21, 42, 71, 76]])>)


In [52]:
print(ds.element_spec)
print(ds2.element_spec)
print(ds3.element_spec)

TensorSpec(shape=(), dtype=tf.int32, name=None)
TensorSpec(shape=(10,), dtype=tf.float32, name=None)
(TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(4,), dtype=tf.int32, name=None))


In [55]:
for e in tf.data.Dataset.range(3):
    print(e)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)


In [64]:
inc_ds = tf.data.Dataset.range(100)
inc_ds2 = tf.data.Dataset.range(100, 200)
comb_ds = tf.data.Dataset.zip((inc_ds, inc_ds2))
batch_ds = comb_ds.batch(10)

In [65]:
list(iter(comb_ds))[:3]

[(<tf.Tensor: shape=(), dtype=int64, numpy=0>,
  <tf.Tensor: shape=(), dtype=int64, numpy=100>),
 (<tf.Tensor: shape=(), dtype=int64, numpy=1>,
  <tf.Tensor: shape=(), dtype=int64, numpy=101>),
 (<tf.Tensor: shape=(), dtype=int64, numpy=2>,
  <tf.Tensor: shape=(), dtype=int64, numpy=102>)]

In [82]:
for e in batch_ds: # this continues until all elements 
    print(e) 
    break

(<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)>, <tf.Tensor: shape=(10,), dtype=int64, numpy=array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109], dtype=int64)>)


In [80]:
for e in batch_ds.take(2): # only take(n), n element within ds.
    print(e)

(<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int64)>, <tf.Tensor: shape=(10,), dtype=int64, numpy=array([100, 101, 102, 103, 104, 105, 106, 107, 108, 109], dtype=int64)>)
(<tf.Tensor: shape=(10,), dtype=int64, numpy=array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19], dtype=int64)>, <tf.Tensor: shape=(10,), dtype=int64, numpy=array([110, 111, 112, 113, 114, 115, 116, 117, 118, 119], dtype=int64)>)


In [83]:
"""
Since each element may be different length, especially in sequential models. tf provide Dataset.padded_batch
function
"""
# dataset = tf.data.Dataset.range(100)
# dataset = dataset.map(lambda x: tf.fill([tf.cast(x, tf.int32)], x))
# dataset = dataset.padded_batch(4, padded_shapes=(None,))

# for batch in dataset.take(2):
#     print(batch.numpy())
#     print()

'\nSince each element may be different length, especially in sequential models. tf provide Dataset.padded_batch\nfunction\n'

Petastorm

In [8]:
rel_path = '../datasets/h&m/transaction_train_sample.parquet'

In [18]:
os.path.join('file://', os.getcwd())

'C:\\Users\\haneu\\Desktop\\PROJECTS\\Finance'

In [32]:
'file://' + os.getcwd()

'file://C:\\Users\\haneu\\Desktop\\PROJECTS\\Finance'

In [33]:
os.path.join(os.getcwd(), 'parquet_files')

'C:\\Users\\haneu\\Desktop\\PROJECTS\\Finance\\parquet_files'

In [34]:
os.path.join("file://", os.path.join(os.getcwd(), 'parquet_files'))

'C:\\Users\\haneu\\Desktop\\PROJECTS\\Finance\\parquet_files'

In [56]:
from petastorm import make_reader, make_batch_reader
from petastorm.tf_utils import make_petastorm_dataset

# filepath = 'file://parquet_files'
filepath = os.path.join("file://", os.path.join(os.getcwd(), 'parquet_files'))
filepath = "file:///Users/haneu/Desktop/PROJECTS/Finance/parquet_files"
print(filepath)
with make_reader(dataset_url=filepath) as reader:
    for row in reader:
        print(row)

In [77]:
with make_batch_reader(dataset_url_or_urls=filepath) as reader:
    dataset = make_petastorm_dataset(reader)
#     for row in reader:
#         print(row)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [81]:
dataset.element_spec

inferred_schema_view(t_dat=TensorSpec(shape=(None,), dtype=tf.string, name=None), customer_id=TensorSpec(shape=(None,), dtype=tf.string, name=None), article_id=TensorSpec(shape=(None,), dtype=tf.int64, name=None), price=TensorSpec(shape=(None,), dtype=tf.float64, name=None), sales_channel_id=TensorSpec(shape=(None,), dtype=tf.int64, name=None))

In [80]:
type(dataset)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter